## Check fMRI Datasets

OpenNeuro is an platform sharing open datasets in neuroscience. The complete WashU fMRI dataset can be found at https://openneuro.org/datasets/ds000243/versions/00001. 


Load packages.

In [3]:
from nilearn import datasets, image, maskers
from nilearn.maskers import NiftiLabelsMasker  # for ROI time-series extraction
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Download a Desikan-Killiany brain atlas.

In [ ]:
import urllib.request

# URL for Desikan-Killiany atlas in MNI152 space (1x1x1 mm resolution)
atlas_url = "https://github.com/neurodata/neuroparc/raw/master/atlases/label/Human/Desikan_space-MNI152NLin6_res-1x1x1.nii.gz"
atlas_path = "Desikan_Killiany_68.nii.gz"
urllib.request.urlretrieve(atlas_url, atlas_path)
print("Atlas downloaded to:", atlas_path)

Load the fMRI dataset.

In [ ]:
fmri_path = "washu_mri/sub-001_task-rest_run-1_bold.nii.gz"  # path to your 4D fMRI NIfTI file
fmri_img = nib.load(fmri_path)
fmri_data = fmri_img.get_fdata()
print("fMRI data shape (x, y, z, time):", fmri_img.shape)


fMRI data shape (x, y, z, time): (64, 64, 32, 480)


Compute the region-averaged fMRI signals.

In [ ]:
# Create a masker for the atlas
masker = NiftiLabelsMasker(labels_img=atlas_path, 
                           standardize=False,  # set True to z-score each time series if desired
                           smoothing_fwhm=None  # e.g., set to 6.0 for 6mm spatial smoothing
                          )

# Extract the mean time series for each atlas region
time_series = masker.fit_transform(fmri_path)
print("Time series array shape:", time_series.shape)

# If you have a list of region names corresponding to the 68 labels:
region_names = [f"Region_{i}" for i in range(1, time_series.shape[1]+1)]
# Create DataFrame and save to CSV
df = pd.DataFrame(time_series, columns=region_names)
df.to_csv("washu_mri/Desikan68_timeseries.csv", index=False)